In [ ]:
# fmt: off
import os
import webbrowser

import branca
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from folium.plugins import (Fullscreen, Geocoder, MeasureControl, MiniMap,
                            TimestampedGeoJson)
from pyprojroot import here
from shapely.geometry import mapping

# fmt: on

In [ ]:
def convert_to_gpdf(df, lat_col="Latitude", long_col="Longitude"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[long_col], df[lat_col]))


def add_folium_times(s: pd.Series, times_prop_name: str) -> list:
    """Utility function to extend times_prop_name column to a list that
    matches the geometry shape - requirement for folium timestampedGeoJSON
    function"""
    return [s[times_prop_name]]  # * len(s.geometry.geoms)


def scale_col(df, col_name, min_val=2, max_val=12):

    srq_root_col = np.sqrt(df[col_name])
    col_min = srq_root_col.min()
    col_max = srq_root_col.max()

    return min_val + ((srq_root_col - col_min) * (max_val - min_val)) / (
        col_max - col_min
    )


def write_tooltip(name, time, tiploc, sheduled, timetabled, percentage):
    # color of columns
    left_col_color = "#0F8243"
    right_col_color = "#EAEAEA"

    # html string, first using the area_name as the title, then adding the
    # modality, time and value to a summary table
    html = (
        """<!DOCTYPE html>
        <html>
        <head>
        <h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(
            name
        )
        + """
        </head>
        <table style="height: 125px; width: 300px;">
        <tbody>
        <tr>
        <td style="background-color: """
        + left_col_color
        + """;"><span style="color: #ffffff;">Day (YYYY-MM-DD)</span></td>
        <td style="width: 150px;background-color: """
        + right_col_color
        + """;">{}</td>""".format(time)
        + """
        </tr>
        <tr>
        <td style="background-color: """
        + left_col_color
        + """;"><span style="color: #ffffff;">TIPLOC code</span></td>
        <td style="width: 150px;background-color: """
        + right_col_color
        + """;">{}</td>""".format(tiploc)
        + """
        </tr>
        <tr>
        <td style="background-color: """
        + left_col_color
        + """;"><span style="color: #ffffff;">No. Scheduled Movements</span></td>
        <td style="width: 150px;background-color: """
        + right_col_color
        + """;">{}</td>""".format(sheduled)
        + """
        </tr>
        <tr>
        <td style="background-color: """
        + left_col_color
        + """;"><span style="color: #ffffff;">No. Timetabled Movements</span></td>
        <td style="width: 150px;background-color: """
        + right_col_color
        + """;">{:.0f}</td>""".format(timetabled)
        + """
        </tr>
        <tr>
        <td style="background-color: """
        + left_col_color
        + """;"><span style="color: #ffffff;">Percentage Running</span></td>
        <td style="width: 150px;background-color: """
        + right_col_color
        + """;">{:.1f}%</td>""".format(percentage)
        + """
        </tr>
        </tbody>
        </table>
        </html>
        """
    )
    return html

In [ ]:
df = pd.read_csv(
    os.path.join(
        here(),
        "output",
        "full_uk_disruption_summary_multiday_start_20220803_14days.csv",
    ),
    index_col=0,
)
df.head()

measure_control = True
mini_map = True
full_screen = True
add_geocoder = True

In [ ]:
gp_df = convert_to_gpdf(df, lat_col="Latitude", long_col="Longitude")
gp_df.head()

In [ ]:
gp_df["times"] = gp_df.apply(add_folium_times, axis=1, args=["date"])
gp_df.head()

In [ ]:
gp_df["radius"] = scale_col(gp_df, "journeys_scheduled", 2, 12)
print(gp_df["radius"].min(), gp_df["radius"].max())
gp_df.head()

In [ ]:
colormap_min = 50
colormap_max = 100
colormap_caption = "Percentage of Scheduled Movements (%)"
colormap = branca.colormap.linear.RdYlBu_05.scale(colormap_min, colormap_max)
colormap.caption = colormap_caption

# def get_colour(s, colormap):

#     pct = s["pct_timetabled_services_running"]

#     if pct <= 50:
#         return colormap(50)
#     elif pct >= 100:
#         return colormap(100)
#     else:
#         return colormap(pct)

# gp_df['colour'] = gp_df.apply(get_colour, axis=1, args=[colormap])
# gp_df.head()

In [ ]:
def get_colour(val):
    if val < 50:
        return "#8b0000"
    elif (val >= 50) & (val < 60):
        return "#ff0000"
    elif (val >= 60) & (val < 70):
        return "#ff0066"
    elif (val >= 70) & (val < 80):
        return "#ff00cc"
    elif (val >= 80) & (val < 90):
        return "#cc00ff"
    elif (val >= 90) & (val < 100):
        return "#6600ff"
    elif val >= 100:
        return "#0000ff"
    else:
        return "#808080"

In [ ]:
from branca.element import MacroElement, Template

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title></title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 10px; bottom: 200px;'>
     
<div class='legend-title'>Percentage of Timetabled<br>Services Running</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#8b0000;opacity:0.5;'></span><50%</li>
    <li><span style='background:#ff0000;opacity:0.5;'></span>[50%, 60%)</li>
    <li><span style='background:#ff0066;opacity:0.5;'></span>[60%, 70%)</li>
    <li><span style='background:#ff00cc;opacity:0.5;'></span>[70%, 80%)</li>
    <li><span style='background:#cc00ff;opacity:0.5;'></span>[80%, 90%)</li>
    <li><span style='background:#6600ff;opacity:0.5;'></span>[90%, 100%)</li>
    <li><span style='background:#0000ff;opacity:0.5;'></span>≥100%</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

In [ ]:
gp_df = gp_df[gp_df["pct_timetabled_services_running"].isna() == False]

In [ ]:
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": mapping(row["geometry"])["type"],
            "coordinates": mapping(row["geometry"])["coordinates"],
        },
        "properties": {
            "times": row["times"],
            "popup": write_tooltip(
                row["Station_Name"],
                row["times"][0],
                row["TIPLOC"],
                row["journeys_scheduled"],
                row["journeys_timetabled"],
                row["pct_timetabled_services_running"],
            ),
            "style": {"color": ""},
            "icon": "circle",
            "iconstyle": {
                "fillColor": get_colour(
                    row["pct_timetabled_services_running"]
                ),  # colormap(row['pct_timetabled_services_running']), # "#0000FF",
                "fillOpacity": 0.5,
                "radius": row["radius"],  # int(row["movt_count"]) * 5,
            },
        },
    }
    for _, row in gp_df.iterrows()
]

In [ ]:
m = folium.Map(
    tiles="openstreetmap",
    max_bounds=True,
)

# colormap.add_to(m)

In [ ]:
# add full screen button if requested
if full_screen:
    m.add_child(Fullscreen())

if mini_map:
    m.add_child(MiniMap())

if add_geocoder:
    m.add_child(Geocoder(add_marker=False, collapsed=True))

# add measuring controls if requested
if measure_control:
    m.add_child(
        MeasureControl(
            primary_length_unit="kilometers",
        )
    )

In [ ]:
# add TimestampGeoJson to the area
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    transition_time=2000,
    period="P1D",
    duration="PT1s",
    max_speed=2,
    date_options="YYYY-MM-DD",
    auto_play=False,
).add_to(m)

# fit view to bounds
m.fit_bounds(m.get_bounds())

In [ ]:
vis_filepath = os.path.join(
    here(),
    "output",
    "timeseries_full_uk_disruption_multiday_start_20220803_14days_coloured_cat.html",
)
m.get_root().add_child(macro)
m.save(vis_filepath)
webbrowser.open("file://" + vis_filepath)

In [ ]:
gp_df[gp_df.TIPLOC == "LEEDS"]